In [13]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [14]:
from general_functions.conncet_s3 import S3Connection
from general_functions.return_account_ids import return_account_ids
from general_functions.sanitize_accout_name import sanitize_account_name


In [15]:
s3 = S3Connection()
buckets = s3.list_buckets()
buckets = [bucket for bucket in buckets if "innkeepr-targeting" in bucket]
buckets

In [16]:
accounts = return_account_ids()

In [19]:
for bucket in buckets:
    tenant = bucket.split("-")[-1]
    print(f"Bucket: {bucket}, Tenant: {tenant}")
    account = [account["name"] for account in accounts if sanitize_account_name(account["name"]) == tenant]
    if len(account) == 0:
        print(f". !!! No account name was found for {bucket}: {tenant}")
        print("")
        account_name = tenant
    elif len(account) != 1:
        raise ValueError(f". Could not find account for {bucket}: {account}")
    else:
        account_name = account[0]
    key_values = [{"Key": "account", "Value": f"{account_name}"},{"Key": "Vendor", "Value": "Training"}] 
    s3.tag_bucket(bucket, key_values)

    print(f". tagged {bucket}: {account_name}")
